In [22]:
def apiToDatabase():
    import pandas as pd 
    import urllib.request
    import json
    import pyodbc
    
    # https://geo.api.gouv.fr/departements/01/communes' pour le département Ain(01)
    
    ch1="https://geo.api.gouv.fr/departements/"
    ch2=input("votre Département : ") # on demande à l'utilisateur de choisir un département. 
    if (len(ch2)==1):#Pour que l api fonctionne il faut que le numéro de département respecte 
        ch2="0"+ch2  #le format /01/, donc on rajoute le 0 de début.
    ch3="/communes"
    url=ch1+ch2+ch3          # On concatine les chaines de caractères afin d'obtenir 
                             #l'URL finale selon le département 
    f= urllib.request.urlopen(url)
    data= f.read().decode("utf-8") #On lit l'API avec la méthode read et l'ncodage UTF-8
    data_dict=json.loads(data) # Mettre les informations dans un dictionnaire 
    dataset = pd.DataFrame(data_dict) # convertir le dictionnaire en dataframe 
    

    #Création de la base de données 
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=LAPTOP-FHV42EPE\SQLEXPRESS;'
                          'Trusted_Connection=yes;')
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute("IF NOT EXISTS " # Vérifier si la base n'existe pas 
                       "("
                       "SELECT name FROM master.dbo.sysdatabases "
                       "WHERE name = N'france'"
                       " )"
                   "CREATE DATABASE [france]") # Créer la base de données 
    
    #créer les tables
    conn = pyodbc.connect('Driver={SQL Server};' #Connexion a la base de données 
                      'Server=LAPTOP-FHV42EPE\SQLEXPRESS;'
                      'Database=france;'
                      'Trusted_Connection=yes;')
    
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute("IF NOT EXISTS (SELECT * FROM sys.Tables where name = 'region')"

                   "CREATE TABLE region" #Créer la table si elle n'existe pas 
                      "("
                           "nom varchar(200),"
                           "code varchar(10),"
                           "codeDepartement varchar(10),"
                           "codeRegion varchar(10),"
                           "codesPostaux varchar(10),"
                           "populationR int"
                     ")")

                 
    #remplir les tables 
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=LAPTOP-FHV42EPE\SQLEXPRESS;'
                          'Database=france;'
                          'Trusted_Connection=yes;')


    cursor = conn.cursor()
    for i in range (len(data_dict)): # boucle poue insérer toutes les données dans la base de données  
        dataseta=[tuple(data_dict[i].values())]
        Table=conv(dataseta)  
        cursor.executemany("INSERT INTO region (nom, code,codeDepartement,codeRegion,codesPostaux,populationR) VALUES (?,?,?,?,?,?)", Table)   
        conn.commit()
    
    return dataset

In [23]:
def conv(df): #Fonction pour convertir le type de certaines données 
              #pour les insérer à la base de donnes
        li = []
        a=df[0][4]
        for tupl in df[0]: 
             li.append(tupl) 
        li[4]=a[0]
        li=[tuple(li)]
        return li 
    

In [25]:
apiToDatabase()

votre Département : 2


,nom,code,codeDepartement,codeRegion,codesPostaux,population
0,Abbécourt,02001,02,32,[02300],496
1,Achery,02002,02,32,[02800],602
2,Acy,02003,02,32,[02200],1008
3,Agnicourt-et-Séchelles,02004,02,32,[02340],179
4,Aguilcourt,02005,02,32,[02190],391
...,...,...,...,...,...,...
795,Wassigny,02830,02,32,[02630],965
796,Watigny,02831,02,32,[02830],377
797,Wiège-Faty,02832,02,32,[02120],208
798,Wimy,02833,02,32,[02500],476
